### Восстановление пропущенных пробелов в тексте с помощью NLP / DL / алгоритма.

In [21]:
import re, math, ast
import torch
import pandas as pd
from datasets import load_dataset, Dataset
from sklearn.metrics import precision_recall_fscore_support
from transformers import BertTokenizerFast, BertForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification
import ast
import numpy as np

In [22]:
# 1. Загружаем датасет
dataset_train = load_dataset("merionum/ru_paraphraser", split="train")

In [23]:
print(dataset_train[1])

{'id': '2', 'id_1': '202', 'id_2': '8158', 'text_1': 'Право полицейских на проникновение в жилище решили ограничить.', 'text_2': 'Правила внесудебного проникновения полицейских в жилище уточнят.', 'class': '0'}


In [24]:
def clean_text(text: str) -> str:
    text = text.lower()
    text = re.sub(r"\s+", " ", text).strip()    # заменяем множественные пробелы
    return text

In [25]:
# 2. Очищаем датасет
clean_dataset_train = dataset_train.map(lambda x: {"text_1": clean_text(x["text_1"])})

In [26]:
# 3. Функция для генерации текста без пробелов и создания меток позиций
def make_training_example(example):
    text = example["text_1"]
    no_space = text.replace(" ", "")

    labels = []
    pos = 0
    for ch in text:
        if ch == " ":
            continue
        if pos == 0:
            labels.append(0)
        else:
            prev_char = text[pos-1]
            labels.append(1 if prev_char == " " else 0)
        pos += 1

    return {"text_no_space": no_space, "text_with_space": text, "labels": labels}

In [27]:
processed_train = clean_dataset_train.map(make_training_example)

In [28]:
# 4. Проверим результат
print("Оригинал:", processed_train[1]["text_with_space"])
print("Без пробелов:", processed_train[1]["text_no_space"])
print("Метки:", processed_train[1]["labels"])

Оригинал: право полицейских на проникновение в жилище решили ограничить
Без пробелов: правополицейскихнапроникновениевжилищерешилиограничить
Метки: [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]


In [29]:
# 5. Сохраняем результат
df = pd.DataFrame(processed_train)
df.to_csv("train_dataset.csv", index=False)
print("Сохранено train_dataset.csv")

Сохранено train_dataset.csv
